In [25]:
!pip install optuna
!pip install keras-tuner
!pip install ray
!pip install scikeras

In [1]:
import numpy as np
import pandas as pd
import optuna
import warnings
import tensorflow as tf
import dill
from IPython.display import display
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras_tuner import RandomSearch
from ray import tune
from ray.train.tensorflow.keras import ReportCheckpointCallback
import keras
from sklearn.exceptions import FitFailedWarning
from sklearn.model_selection import KFold, cross_val_score, cross_validate, RandomizedSearchCV, train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPClassifier, MLPRegressor
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler

In [42]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FitFailedWarning)

In [2]:
clf_df = pd.read_csv("../data/asteroids_processed.csv")
reg_df = pd.read_csv("../data/cars_processed.csv")

In [4]:
clf_df = pd.read_csv("asteroids_processed.csv")
reg_df = pd.read_csv("cars_processed.csv")

In [16]:
X_clf = clf_df.drop(['hazardous'], axis=1)
y_clf = clf_df['hazardous']

In [17]:
oversample = SMOTE()
X_clf, y_clf = oversample.fit_resample(X_clf, y_clf)

In [18]:
tree = DecisionTreeClassifier().fit(X_clf, y_clf)

In [19]:
rfe = RFE(estimator=tree, n_features_to_select=3, step=1).fit(X_clf, y_clf)
X_clf = pd.DataFrame(rfe.transform(X_clf), columns=rfe.get_feature_names_out())
X_clf.head()

,est_diameter_max,relative_velocity,miss_distance
0,35,56014,1024332
1,68,7864,32681860
2,124,55257,65386361
3,43,41531,12607957
4,311,67639,71305897


In [20]:
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)

In [21]:
scaler = StandardScaler()
X_clf_train = scaler.fit_transform(X_clf_train)
X_clf_test = scaler.transform(X_clf_test)

In [3]:
X_reg = reg_df.drop(['price_usd'], axis=1)
y_reg = reg_df['price_usd']

In [4]:
tree = DecisionTreeRegressor().fit(X_reg, y_reg)

In [5]:
rfe = RFE(estimator=tree, n_features_to_select=13, step=1).fit(X_reg, y_reg)
X_reg = pd.DataFrame(rfe.transform(X_reg), columns=rfe.get_feature_names_out())
X_reg.head()

,manufacturer_name,model_name,color,odometer_value,year_produced,engine_fuel,engine_capacity,body_type,number_of_photos,up_counter,duration_listed,features_count,drivetrain_front
0,45.0,763.0,8.0,190000.0,2010.0,2.0,2.5,10.0,9.0,13.0,16.0,7.0,0.0
1,45.0,664.0,0.0,280000.0,2001.0,2.0,2.5,10.0,14.0,7.0,7.0,4.0,0.0
2,45.0,664.0,8.0,350000.0,2004.0,2.0,2.5,8.0,7.0,29.0,73.0,3.0,0.0
3,45.0,763.0,4.0,179000.0,2010.0,2.0,2.5,10.0,17.0,33.0,87.0,9.0,0.0
4,45.0,519.0,8.0,571317.0,1999.0,2.0,2.5,10.0,8.0,11.0,43.0,4.0,0.0


In [6]:
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_reg_train = scaler.fit_transform(X_reg_train)
X_reg_test = scaler.transform(X_reg_test)

In [8]:
clf_table = pd.DataFrame(columns=[
    'name', 'solver', 'epochs',
    'test_accuracy', 'test_precision', 'test_recall', 'test_F1',
    'train_accuracy', 'train_precision', 'train_recall', 'train_F1'
])
reg_table = pd.DataFrame(columns=[
    'name', 'solver', 'epochs',
    'test_MAE', 'test_MSE', 'test_RMSE', 'test_MAPE', 'test_R2',
    'train_MAE', 'train_MSE', 'train_RMSE', 'train_MAPE', 'train_R2'
])

In [12]:
def print_clf_metrics(model):
    y_test_pred = model.predict(X_clf_test)
    print('Accuracy:', round(accuracy_score(y_clf_test, y_test_pred), 4))
    print('Precision:', round(precision_score(y_clf_test, y_test_pred), 4))
    print('Recall:', round(recall_score(y_clf_test, y_test_pred), 4))
    print('F1-score:', round(f1_score(y_clf_test, y_test_pred), 2))
    print('ROC AUC score:', round(roc_auc_score(y_clf_test, y_test_pred), 4))
    print('Confusion matrix: \n', confusion_matrix(y_clf_test, y_test_pred))

In [11]:
def add_clf_metrics_to_table(model, name, solver, epochs):
    y_test_pred = model.predict(X_clf_test)
    y_train_pred = model.predict(X_clf_train)

    clf_table.loc[len(clf_table)] = {
        'name': name,
        'solver': solver,
        'epochs': epochs,
        'test_accuracy': round(accuracy_score(y_clf_test, y_test_pred), 4),
        'test_precision': round(precision_score(y_clf_test, y_test_pred), 4),
        'test_recall': round(recall_score(y_clf_test, y_test_pred), 4),
        'test_F1': round(f1_score(y_clf_test, y_test_pred), 2),
        'train_accuracy': round(accuracy_score(y_clf_train, y_train_pred), 4),
        'train_precision': round(precision_score(y_clf_train, y_train_pred), 4),
        'train_recall': round(recall_score(y_clf_train, y_train_pred), 4),
        'train_F1': round(f1_score(y_clf_train, y_train_pred), 2)
    }

In [9]:
def print_reg_metrics(model):
    y_test_pred = model.predict(X_reg_test)
    print(f'MAE: {round(mean_absolute_error(y_reg_test, y_test_pred), 4)}')
    print(f'MSE: {round(mean_squared_error(y_reg_test, y_test_pred), 4)}')
    print(f'RMSE: {round(np.sqrt(mean_squared_error(y_reg_test, y_test_pred)), 4)}')
    print(f'MAPE: {round(mean_absolute_percentage_error(y_reg_test, y_test_pred), 4)}')
    print(f'R^2: {round(r2_score(y_reg_test, y_test_pred), 2)}')

In [10]:
def add_reg_metrics_to_table(model, name, solver, epochs):
    y_test_pred = model.predict(X_reg_test)
    y_train_pred = model.predict(X_reg_train)

    reg_table.loc[len(reg_table)] = {
        'name': name,
        'solver': solver,
        'epochs': epochs,
        'test_MAE': round(mean_absolute_error(y_reg_test, y_test_pred), 4),
        'test_MSE': round(mean_squared_error(y_reg_test, y_test_pred), 4),
        'test_RMSE': round(np.sqrt(mean_squared_error(y_reg_test, y_test_pred)), 4),
        'test_MAPE': round(mean_absolute_percentage_error(y_reg_test, y_test_pred), 4),
        'test_R2': round(r2_score(y_reg_test, y_test_pred), 2),
        'train_MAE': round(mean_absolute_error(y_reg_train, y_train_pred), 4),
        'train_MSE': round(mean_squared_error(y_reg_train, y_train_pred), 4),
        'train_RMSE': round(np.sqrt(mean_squared_error(y_reg_train, y_train_pred)), 4),
        'train_MAPE': round(mean_absolute_percentage_error(y_reg_train, y_train_pred), 4),
        'train_R2': round(r2_score(y_reg_train, y_train_pred), 2)
    }

In [60]:
parameters = {
    'batch_size': [32, 64, 128, 256],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'activation': ['logistic', 'tanh', 'relu', 'identity'],
    'alpha': [1e-6, 1e-4, 0.01, 0.05, 0.1]
}

In [61]:
rs = RandomizedSearchCV(MLPClassifier(), parameters, n_jobs=-1).fit(X_clf_train, y_clf_train)
rs.best_params_

{'learning_rate_init': 0.01,
 'batch_size': 128,
 'alpha': 1e-06,
 'activation': 'relu'}

In [62]:
model = MLPClassifier(solver='adam', **rs.best_params_).fit(X_clf_train, y_clf_train)
add_clf_metrics_to_table(model, 'MLPClassifier', 'adam', model.n_iter_)
print_clf_metrics(model)

Accuracy: 0.9123
Precision: 0.8563
Recall: 0.9907
F1-score: 0.92
ROC AUC score: 0.9125
Confusion matrix: 
 [[12527  2490]
 [  139 14838]]


In [63]:
model = MLPClassifier(solver='sgd', **rs.best_params_).fit(X_clf_train, y_clf_train)
add_clf_metrics_to_table(model, 'MLPClassifier', 'sgd', model.n_iter_)
print_clf_metrics(model)

Accuracy: 0.9127
Precision: 0.8581
Recall: 0.9886
F1-score: 0.92
ROC AUC score: 0.9128
Confusion matrix: 
 [[12569  2448]
 [  170 14807]]


In [64]:
model = MLPClassifier(solver='lbfgs', **rs.best_params_).fit(X_clf_train, y_clf_train)
add_clf_metrics_to_table(model, 'MLPClassifier', 'lbfgs', model.n_iter_)
print_clf_metrics(model)

Accuracy: 0.9136
Precision: 0.8588
Recall: 0.9897
F1-score: 0.92
ROC AUC score: 0.9137
Confusion matrix: 
 [[12580  2437]
 [  154 14823]]


In [65]:
clf_table

,name,solver,epochs,test_accuracy,test_precision,test_recall,test_F1,train_accuracy,train_precision,train_recall,train_F1
0,MLPClassifier,adam,27,0.9123,0.8563,0.9907,0.92,0.9134,0.8582,0.9907,0.92
1,MLPClassifier,sgd,44,0.9127,0.8581,0.9886,0.92,0.9137,0.8599,0.9884,0.92
2,MLPClassifier,lbfgs,200,0.9136,0.8588,0.9897,0.92,0.9149,0.8606,0.9903,0.92


In [66]:
rs = RandomizedSearchCV(MLPRegressor(), parameters, n_jobs=-1).fit(X_reg_train, y_reg_train)
rs.best_params_

{'learning_rate_init': 0.1,
 'batch_size': 32,
 'alpha': 1e-06,
 'activation': 'logistic'}

In [67]:
model = MLPRegressor(solver='adam', **rs.best_params_).fit(X_reg_train, y_reg_train)
add_reg_metrics_to_table(model, 'MLPRegressor', 'adam', model.n_iter_)
print_reg_metrics(model)

MAE: 1018.7154
MSE: 2128601.315
RMSE: 1458.9727
MAPE: 0.3469
R^2: 0.88


In [69]:
parameters = {
    'batch_size': [32, 64, 128, 256],
    'learning_rate_init': [0.0001, 0.001, 0.01],
    'activation': ['logistic', 'tanh', 'relu', 'identity'],
    'alpha': [1e-4, 0.01, 0.05, 0.1]
}

In [70]:
rs_sgd = RandomizedSearchCV(MLPRegressor(solver='sgd'), parameters, n_jobs=-1).fit(X_reg_train, y_reg_train)
rs_sgd.best_params_

{'learning_rate_init': 0.0001,
 'batch_size': 256,
 'alpha': 0.1,
 'activation': 'tanh'}

In [71]:
model = MLPRegressor(solver='sgd', **rs_sgd.best_params_).fit(X_reg_train, y_reg_train)
add_reg_metrics_to_table(model, 'MLPRegressor', 'sgd', model.n_iter_)
print_reg_metrics(model)

MAE: 1169.6698
MSE: 2700480.5182
RMSE: 1643.3139
MAPE: 0.3878
R^2: 0.85


In [72]:
model = MLPRegressor(solver='lbfgs', **rs.best_params_).fit(X_reg_train, y_reg_train)
add_reg_metrics_to_table(model, 'MLPRegressor', 'lbfgs', model.n_iter_)
print_reg_metrics(model)

MAE: 1061.2463
MSE: 2239442.7418
RMSE: 1496.4768
MAPE: 0.3735
R^2: 0.87


In [73]:
reg_table

,name,solver,epochs,test_MAE,test_MSE,test_RMSE,test_MAPE,test_R2,train_MAE,train_MSE,train_RMSE,train_MAPE,train_R2
0,MLPRegressor,adam,152,1018.7154,2.128601e+06,1458.9727,0.3469,0.88,968.2330,1.961305e+06,1400.4661,0.7845,0.89
1,MLPRegressor,sgd,55,1169.6698,2.700481e+06,1643.3139,0.3878,0.85,1137.8213,2.654062e+06,1629.1292,0.8360,0.85
2,MLPRegressor,lbfgs,200,1061.2463,2.239443e+06,1496.4768,0.3735,0.87,973.9970,1.935182e+06,1391.1081,0.8411,0.89


In [22]:
X_clf_train = X_clf_train.copy()
y_clf_train = y_clf_train.copy()
X_clf_test = X_clf_test.copy()
y_clf_test = y_clf_test.copy()

X_reg_train = X_reg_train.copy()
y_reg_train = y_reg_train.copy()
X_reg_test = X_reg_test.copy()
y_reg_test = y_reg_test.copy()

In [75]:
def objective(trial):
    solver = trial.suggest_categorical('solver', ['lbfgs', 'adam', 'sgd'])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate_init = trial.suggest_categorical('learning_rate_init', [0.001, 0.01, 0.1])
    activation = trial.suggest_categorical('activation', ['logistic', 'tanh', 'relu', 'identity'])
    alpha = trial.suggest_categorical('alpha', [1e-6, 1e-4, 0.01, 0.05, 0.1])

    model = MLPClassifier(
        solver=solver,
        batch_size=batch_size,
        learning_rate_init=learning_rate_init,
        activation=activation,
        alpha=alpha
    )
    model.fit(X_clf_train, y_clf_train)
    y_pred = model.predict(X_clf_test)
    return accuracy_score(y_clf_test, y_pred)

In [76]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
best_model = MLPClassifier(**best_params).fit(X_clf_train, y_clf_train)
print_clf_metrics(best_model)
best_params

Accuracy: 0.914
Precision: 0.8595
Recall: 0.9896
F1-score: 0.92
ROC AUC score: 0.9142
Confusion matrix: 
 [[12595  2422]
 [  156 14821]]


{'solver': 'lbfgs',
 'batch_size': 64,
 'learning_rate_init': 0.001,
 'activation': 'relu',
 'alpha': 0.05}

In [23]:
def objective(trial):
    solver = trial.suggest_categorical('solver', ['lbfgs', 'adam', 'sgd'])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    learning_rate_init = trial.suggest_categorical('learning_rate_init', [0.001, 0.01, 0.1])
    activation = trial.suggest_categorical('activation', ['logistic', 'tanh', 'relu', 'identity'])
    alpha = trial.suggest_categorical('alpha', [1e-6, 1e-4, 0.01, 0.05, 0.1])

    model = MLPRegressor(
        solver=solver,
        batch_size=batch_size,
        learning_rate_init=learning_rate_init,
        activation=activation,
        alpha=alpha
    )
    model.fit(X_reg_train, y_reg_train)
    y_pred = model.predict(X_reg_test)
    return mean_squared_error(y_reg_test, y_pred)

In [24]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
best_model = MLPRegressor(**best_params).fit(X_reg_train, y_reg_train)
print_reg_metrics(best_model)
best_params

[I 2025-06-04 12:02:13,092] A new study created in memory with name: no-name-7bd5153f-33a1-42ab-a808-3f95adcfb7d0
c:\Users\admin\OneDrive\Рабочий стол\ML&BD\venv\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
c:\Users\admin\OneDrive\Рабочий стол\ML&BD\venv\Lib\site-packages\sklearn\utils\extmath.py:208: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
c:\Users\admin\OneDrive\Рабочий стол\ML&BD\venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[W 2025-06-04 12:03:36,666] Trial 0 failed with parameters: {'solver': 'sgd', 'batch_size': 32, 'learning_rate_init': 0.1, 'activation': 'identity', 'alpha': 0.05} because of the following error: ValueError('Solver produced non-finite parameter weights. The input data may contain large values and need to be pre

ValueError: Solver produced non-finite parameter weights. The input data may contain large values and need to be preprocessed.

In [ ]:
space = {
    'solver': hp.choice('solver', ['lbfgs', 'adam', 'sgd']),
    'batch_size': hp.choice('batch_size', [32, 64, 128, 256]),
    'learning_rate_init': hp.choice('learning_rate_init', [0.001, 0.01, 0.1]),
    'activation': hp.choice('activation', ['logistic', 'tanh', 'relu', 'identity']),
    'alpha': hp.choice('alpha', [1e-6, 1e-4, 0.01, 0.05, 0.1])
}

In [ ]:
def objective(params):
    model = MLPClassifier(**params)
    model.fit(X_clf_train, y_clf_train)
    y_pred = model.predict(X_clf_test)
    score = accuracy_score(y_clf_test, y_pred)
    return {'loss': -score, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best_params = fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

best_model = MLPClassifier(**best_params).fit(X_clf_train, y_clf_train)
print_clf_metrics(best_model)
best_params

NameError: name 'objective' is not defined

In [ ]:
def objective(params):
    model = MLPRegressor(**params) 
    model.fit(X_reg_train, y_reg_train) 
    y_pred = model.predict(X_reg_test) 
    score = mean_squared_error(y_reg_test, y_pred)
    return {'loss': score, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best_params = fmin(objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

best_model = MLPRegressor(**best_params).fit(X_reg_train, y_reg_train)
print_reg_metrics(best_model)
best_params

In [ ]:
def make_clf_model(optimizer_name, **kwargs):
    activation = kwargs.get("activation")
    l2 = kwargs.get("l2")
    units_0 = kwargs.get("units_0")
    units_1 = kwargs.get("units_1")
    dropout = kwargs.get("dropout")
    lr = kwargs.get("lr")

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units_0, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(l2),
                              input_shape=(X_clf_train.shape[1],)),
        tf.keras.layers.Dense(units_1, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(l2)),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def make_reg_model(optimizer_name, **kwargs):
    activation = kwargs.get("activation")
    l2 = kwargs.get("l2")
    units_0 = kwargs.get("units_0")
    units_1 = kwargs.get("units_1")
    dropout = kwargs.get("dropout")
    lr = kwargs.get("lr")

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units_0, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(l2),
                              input_shape=(X_reg_train.shape[1],)),
        tf.keras.layers.Dense(units_1, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(l2)),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

    model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])
    return model

In [ ]:
def objective(trial):
    activation = trial.suggest_categorical("activation", ["sigmoid", "tanh", "relu"])
    l2_alpha = trial.suggest_categorical("l2", [1e-6, 1e-4, 0.01, 0.05, 0.1])
    lr = trial.suggest_categorical("lr", [1e-4, 1e-3, 1e-2])
    units_0 = trial.suggest_categorical("units_0", [8, 16, 32, 64])
    units_1 = trial.suggest_categorical("units_1", [8, 16, 32, 64])
    dropout_rate = trial.suggest_float("dropout", 0.0, 0.3, step=0.05)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units_0, activation=activation,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_alpha),
                              input_shape=(X_clf_train.shape[1],)),
        tf.keras.layers.Dense(units_1, activation=activation,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_alpha)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

    history = model.fit(
        X_clf_train, y_clf_train,
        validation_split=0.2,
        epochs=50,
        verbose=0,
        callbacks=[stop_early]
    )

    return max(history.history["val_accuracy"])

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_params

{'activation': 'relu',
 'optimizer': 'sgd',
 'l2': 1e-06,
 'lr': 0.01,
 'units_0': 16,
 'units_1': 32,
 'dropout': 0.0}

In [ ]:
best_params_prefixed = {f"model__{k}": v for k, v in best_params.items()}

model = make_clf_model(optimizer_name = 'adam', **best_params_prefixed)
display(tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, dpi=108))

In [ ]:
model.summary()

In [ ]:
model = KerasClassifier(model=make_clf_model, optimizer_name = 'adam', **best_params_prefixed, epochs=50)
print_clf_metrics(model)

In [ ]:
def objective(trial):
    activation = trial.suggest_categorical("activation", ["sigmoid", "tanh", "relu"])
    l2_alpha = trial.suggest_categorical("l2", [1e-6, 1e-4, 0.01, 0.05, 0.1])
    lr = trial.suggest_categorical("lr", [1e-4, 1e-3, 1e-2])
    units_0 = trial.suggest_categorical("units_0", [8, 16, 32, 64])
    units_1 = trial.suggest_categorical("units_1", [8, 16, 32, 64])
    dropout_rate = trial.suggest_float("dropout", 0.0, 0.3, step=0.05)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units_0, activation=activation,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_alpha),
                              input_shape=(X_reg_train.shape[1],)),
        tf.keras.layers.Dense(units_1, activation=activation,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_alpha)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(1, activation="linear")
    ])

    model.compile(optimizer=optimizer, loss="mse")
    stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

    history = model.fit(
        X_reg_train, y_reg_train,
        validation_split=0.2,
        epochs=50,
        verbose=0,
        callbacks=[stop_early]
    )

    return min(history.history["val_loss"])

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

best_params = study.best_params
best_params

In [ ]:
best_params_prefixed = {f"model__{k}": v for k, v in best_params.items()}

model = make_reg_model(optimizer_name = 'adam', **best_params_prefixed)
display(tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True, dpi=108))

In [ ]:
model.summary()

In [ ]:
model = KerasRegressor(model=make_reg_model, optimizer_name = 'adam', **best_params_prefixed, epochs=50)
print_reg_metrics(model)

In [ ]:
def build_model(hp):
    activation = hp.Choice("activation", ['sigmoid', 'tanh', 'relu'])
    optimizer_name = hp.Choice("optimizer", ["adam", "rmsprop", "sgd"])
    l2_alpha = hp.Choice("l2", [1e-6, 1e-4, 0.01, 0.05, 0.1])
    lr = hp.Choice("lr", [1e-4, 1e-3, 1e-2])

    units_0 = hp.Choice("units_0", [8, 16, 32, 64])
    units_1 = hp.Choice("units_1", [8, 16, 32, 64])
    dropout = hp.Float("dropout", 0.0, 0.3, step=0.05)

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(
            units_0,
            activation=activation,
            kernel_regularizer=tf.keras.regularizers.l2(l2_alpha),
            input_shape=(X_clf_train.shape[1],)
        ),
        tf.keras.layers.Dense(
            units_1,
            activation=activation,
            kernel_regularizer=tf.keras.regularizers.l2(l2_alpha)
        ),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model


In [ ]:
tuner = RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=30,
    directory="my_tuner_runs",
    project_name="clf_search_v1"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

tuner.search(X_clf_train, y_clf_train,
             validation_split=0.2,
             epochs=50,
             callbacks=[stop_early],
             verbose=1)

Trial 30 Complete [00h 08m 05s]
val_loss: 0.2534012496471405

Best val_loss So Far: 0.228085458278656
Total elapsed time: 02h 51m 13s


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hp.values

{'activation': 'relu',
 'optimizer': 'sgd',
 'l2': 1e-06,
 'lr': 0.01,
 'units_0': 64,
 'units_1': 32,
 'dropout': 0.25}

In [ ]:
def build_model(config):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(
        config["units_0"],
        activation=config["activation"],
        kernel_regularizer=tf.keras.regularizers.l2(config["l2"]),
        input_shape=(3,)
    ))
    model.add(tf.keras.layers.Dense(
        config["units_1"],
        activation=config["activation"],
        kernel_regularizer=tf.keras.regularizers.l2(config["l2"])
    ))
    model.add(tf.keras.layers.Dropout(config["dropout"]))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    if config["optimizer"] == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=config["lr"])
    elif config["optimizer"] == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=config["lr"])
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=config["lr"])

    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def train_model(config):
    model = build_model(config)
    model.fit(
        X_clf_train, y_clf_train,
        validation_split=0.2,
        epochs=50,
        verbose=0,
        callbacks=[ReportCheckpointCallback(metrics={"val_loss": "val_loss", "accuracy": "accuracy"})]
    )

In [ ]:
search_space = {
    "activation": tune.choice(["relu", "tanh", "sigmoid"]),
    "optimizer": tune.choice(["adam", "rmsprop", "sgd"]),
    "l2": tune.choice([1e-6, 1e-4, 0.01, 0.05, 0.1]),
    "lr": tune.choice([1e-4, 1e-3, 1e-2]),
    "units_0": tune.choice([8, 16, 32, 64]),
    "units_1": tune.choice([8, 16, 32, 64]),
    "dropout": tune.uniform(0.0, 0.3)
}

In [ ]:
tuner = tune.Tuner(
    train_model,
    param_space=search_space,
    tune_config=tune.TuneConfig(
        metric="val_loss",
        mode="min",
        num_samples=10
    )
)

results = tuner.fit()

2025-06-02 10:23:56,043	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


+--------------------------------------------------------------------+
| Configuration for experiment     train_model_2025-06-02_10-23-56   |
+--------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator             |
| Scheduler                        FIFOScheduler                     |
| Number of trials                 10                                |
+--------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_2025-06-02_10-23-56

Trial status: 10 PENDING
Current time: 2025-06-02 10:23:56. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+-----------------------------------------------------------------------------------------------------------------------+
| Trial name                status     activation     optimizer         l2       lr     units_0     units_1     dropout |
+----------------------

(pid=2952) 2025-06-02 10:24:00.528939: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2952) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=2952) E0000 00:00:1748859840.564095    2952 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=2952) E0000 00:00:1748859840.574791    2952 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Trial train_model_b065d_00001 started with configuration:
+--------------------------------------------------+
| Trial train_model_b065d_00001 config             |
+--------------------------------------------------+
| activation                               sigmoid |
| dropout                                  0.19758 |
| l2                                        0.0001 |
| lr                                        0.0001 |
| optimizer                                rmsprop |
| units_0                                        8 |
| units_1                                       64 |
+--------------------------------------------------+

Trial train_model_b065d_00000 started with configuration:
+--------------------------------------------------+
| Trial train_model_b065d_00000 config             |
+--------------------------------------------------+
| activation                               sigmoid |
| dropout                                  0.05604 |
| l2                              

(train_model pid=2952) /usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
(train_model pid=2952)   super().__init__(activity_regularizer=activity_regularizer, **kwargs)
(pid=2980) 2025-06-02 10:24:00.530575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2980) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=2980) E0000 00:00:1748859840.568337    2980 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=2980) E0000 00:00:1748859840.578251    2980 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to r

...


Trial train_model_b065d_00008 completed after 50 iterations at 2025-06-02 11:05:45. Total running time: 41min 49s
+------------------------------------------------------------+
| Trial train_model_b065d_00008 result                       |
+------------------------------------------------------------+
| checkpoint_dir_name                      checkpoint_000049 |
| time_this_iter_s                                  10.21722 |
| time_total_s                                     501.12712 |
| training_iteration                                      50 |
| accuracy                                           0.50053 |
| val_loss                                           0.69315 |
+------------------------------------------------------------+


(train_model pid=18579) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_model_2025-06-02_10-23-56/train_model_b065d_00009_9_activation=sigmoid,dropout=0.0313,l2=0.0500,lr=0.0001,optimizer=rmsprop,units_0=32,units_1=8_2025-06-02_10-23-56/checkpoint_000046) [repeated 2x across cluster]
(train_model pid=18579) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_model_2025-06-02_10-23-56/train_model_b065d_00009_9_activation=sigmoid,dropout=0.0313,l2=0.0500,lr=0.0001,optimizer=rmsprop,units_0=32,units_1=8_2025-06-02_10-23-56/checkpoint_000047)



Trial status: 9 TERMINATED | 1 RUNNING
Current time: 2025-06-02 11:06:01. Total running time: 42min 5s
Logical resource usage: 1.0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
Current best trial: b065d_00004 with val_loss=0.22828324139118195 and params={'activation': 'relu', 'optimizer': 'adam', 'l2': 1e-06, 'lr': 0.0001, 'units_0': 16, 'units_1': 32, 'dropout': 0.24022020509911002}
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                status       activation     optimizer         l2       lr     units_0     units_1     dropout     iter     total time (s)     val_loss     accuracy |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_model_b065d_00009   RUNNING      sigmoid        rmsprop       0.05   

(train_model pid=18579) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_model_2025-06-02_10-23-56/train_model_b065d_00009_9_activation=sigmoid,dropout=0.0313,l2=0.0500,lr=0.0001,optimizer=rmsprop,units_0=32,units_1=8_2025-06-02_10-23-56/checkpoint_000048)
(train_model pid=18579) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/train_model_2025-06-02_10-23-56/train_model_b065d_00009_9_activation=sigmoid,dropout=0.0313,l2=0.0500,lr=0.0001,optimizer=rmsprop,units_0=32,units_1=8_2025-06-02_10-23-56/checkpoint_000049)
2025-06-02 11:06:10,302	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_2025-06-02_10-23-56' in 0.0166s.



Trial train_model_b065d_00009 completed after 50 iterations at 2025-06-02 11:06:10. Total running time: 42min 14s
+------------------------------------------------------------+
| Trial train_model_b065d_00009 result                       |
+------------------------------------------------------------+
| checkpoint_dir_name                      checkpoint_000049 |
| time_this_iter_s                                   6.14288 |
| time_total_s                                     475.69371 |
| training_iteration                                      50 |
| accuracy                                           0.50228 |
| val_loss                                           0.69315 |
+------------------------------------------------------------+

Trial status: 10 TERMINATED
Current time: 2025-06-02 11:06:10. Total running time: 42min 14s
Logical resource usage: 1.0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
Current best trial: b065d_00004 with val_loss=0.22828324139118195 and params={'activat

In [ ]:
best_result = results.get_best_result(metric="val_loss", mode="min")
best_result.config

{'activation': 'relu',
 'optimizer': 'adam',
 'l2': 1e-06,
 'lr': 0.0001,
 'units_0': 16,
 'units_1': 32,
 'dropout': 0.24022020509911002}

In [79]:
dill.dump_session('session.pkl')

In [ ]:
dill.load_session('session.pkl')